## AUTOMATIZACION TABLA_PERIODOS BALANCES DE BANCOS

In [27]:
from utils.functions_ia import *
from gspread_formatting  import *

#me me conecto a la spreadsheet y agarro sus datos
key = "1Ed0M09j0yk2LCLU0sBmAUximavxcZoesNmDd1w25TC8"
sheet_name = "PERIODOS"
gc = connect_spreadsheets()
sh = gc.open_by_key(key)
worksheet = sh.worksheet(sheet_name)

wks_data = worksheet.get_all_values() #es una lista de listas de cada fila del spreadsheet

#me conecto a la base de datos
cnxn = connect_database() #genero la conexion con la db

#inserto los datos de la spreadsheet en la db
cursor = cnxn.cursor()

fmt = cellFormat(backgroundColor=color(0, 1, 0) )#color de la celda

#inserto fila por fila en la base de datos
for i in range(1,len(wks_data)):
    query = f"BEGIN IF NOT EXISTS (SELECT * FROM ACTIVE.DBO.PERIODOS_TABLEROS WHERE PERIODO = '{wks_data[i][3]}') BEGIN INSERT INTO ACTIVE.DBO.PERIODOS_TABLEROS (IDCLIENTE, MES, AÑO, PERIODO, FINICIO, FFIN) VALUES {tuple(wks_data[i])} END END"
    try:
        cursor.execute(query)
        cnxn.commit()
        format_cell_range(worksheet, f'{i+1}', fmt)
        print('Insert Finish!')
    except: 
        print("No se pudo insertar")
cnxn.close()


Insert Finish!
Insert Finish!
Insert Finish!


## AUTOMATIZACION RELLENO REFIS


In [56]:
from utils.functions_ia import *
import time

#me me conecto a la spreadsheet y agarro sus datos
key = "1Ed0M09j0yk2LCLU0sBmAUximavxcZoesNmDd1w25TC8"
sheet_name = "REFI (PAG_UNIF)"
sheet_name2 = "REFI(HISTORICO)"
gc = connect_spreadsheets()
sh = gc.open_by_key(key)
worksheet = sh.worksheet(sheet_name)
worksheet2 = sh.worksheet(sheet_name2)

wks_data = worksheet.get_all_values() #es una lista de listas de cada fila del spreadsheet

#me conecto a la base de datos
cnxn = connect_database() #genero la conexion con la db

#inserto los datos de la spreadsheet en la db
cursor = cnxn.cursor()

#agarro el ultimo id pago 
query_ = "SELECT MAX(ID_PAGO) FROM ACTIVE.DBO.[PAGOS_UNIFICADOS] WHERE TIPO_PAGO ='REFI'"
max_id_pago = cursor.execute(query_).fetchall()[0][0] #el ultimo id_pagos


#me conecto a la base de datos
cols = wks_data[0]

#supervisores  gente a la que se le envia el mail en caso exitoso
supervisores = ['vanessa.fondado@mopc.cc','nicolas.rodriguez@mopc.cc','luis.castillo@mopc.cc','esteban.encina@mopc.cc','Nicolas.Caruso@mopc.cc']


resultados= []

#inserto fila por fila en la base de datos
n = 0
for i in range(1,len(wks_data)):
    values = wks_data[i]
    values[cols.index('FECHA_CARGA')] = time.strftime("%Y-%d-%m %H:%M:%S", time.localtime()) # fecha carga el tiempo actual
    values[cols.index('ID_PAGO')] = max_id_pago + i #le iserto el id-pago
    values[cols.index('EMPRESA')] = 'MO&PC' #este valor es fijo
    values[cols.index('TIPO_PAGO')] = 'REFI' #este valor es fijo
    print(values)
    fecha_pago = values[cols.index("FECHA_PAGO")]
    idmoroso = values[cols.index("IDMOROSO")]
    query = f"BEGIN IF NOT EXISTS (SELECT * FROM ACTIVE.DBO.PAGOS_UNIFICADOS WHERE TIPO_PAGO ='REFI' AND FECHA_PAGO = '{fecha_pago}' AND IDMOROSO = '{idmoroso}') BEGIN INSERT INTO ACTIVE.DBO.PAGOS_UNIFICADOS (FUENTE_PAGO, ID_PAGO, RUC, FECHA_PAGO, FECHA_REMESA, DESTINATARIO, IDCLIENTE, IDMOROSO, MONTO, HONORARIOS, FILTROA, FILTROB, FILTROC,DEUDA_TOTAL, DEUDA_VENCIDA, TIPO_PAGO, FECHA_CARGA, EMPRESA) VALUES {tuple(values)} END END"
    try:
        print(n)
        cursor.execute(query)
        cnxn.commit()
        print('Insert Finish!')
        #worksheet2.resize(1)
        worksheet2.append_row(values)
        worksheet.delete_row(i+1 - n)
        n = n+1
        resultados.append(1)
    except: 
        print("No se pudo insertar")
        resultados.append(0)

print(resultados)
# me fijo si todas las filas fueron insertadas correctamente. Si no, se envia un mail automatico avisando
if sum(resultados) == len(wks_data) - 1:
            send_mail(subject = "REFIS: carga correcta",
        body = "Estimados, <br> Las REFIS fueron cargadas correctamente a las base de datos.<br><br>  Este correo se envía con frecuencias semanal y es automático. <br> Cualquier duda o consulta, comunicarse con ia@mopc.cc <br> Atte. <br> Equipo de Inteligencia Artificial" ,
receivers=['lulikad@hotmail.com'], sender=None)
else: 
    send_mail(subject = "REFIS: error",
        body = "Estimados, <br> Las REFIS no fueron cargadas correctamente. La<br><br>  Este correo se envía con frecuencias semanal y es automático. <br> Cualquier duda o consulta, comunicarse con ia@mopc.cc <br> Atte. <br> Equipo de Inteligencia Artificial" ,
receivers=['lulikad@hotmail.com'] , sender=None)

cnxn.close()

[]


In [6]:
wks_data[1][7]
wks_data[0]

 (FUENTE_PAGO, ID_PAGO, RUC, FECHA_PAGO, FECHA_REMESA, DESTINATARIO, IDCLIENTE, IDMOROSO, MONTO, HONORARIOS, FILTROA, FILTROB, FILTROC,DEUDA_TOTAL, DEUDA_VENCIDA, TIPO_PAGO, FECHA_CARGA, EMPRESA)

['FUENTE_PAGO',
 'ID_PAGO',
 'RUC',
 'FECHA_PAGO',
 'FECHA_REMESA',
 'DESTINATARIO',
 'IDCLIENTE',
 'IDMOROSO',
 'MONTO',
 'HONORARIOS',
 'FILTROA',
 'FILTROB',
 'FILTROC',
 'DEUDA_TOTAL',
 'DEUDA_VENCIDA',
 'TIPO_PAGO',
 'FECHA_CARGA',
 'EMPRESA']

In [2]:
from utils.functions_ia import *
import time
from gspread_formatting  import *
from datetime import datetime
#me me conecto a la spreadsheet y agarro sus datos
key = "1Ed0M09j0yk2LCLU0sBmAUximavxcZoesNmDd1w25TC8"
sheet_name = "REFI (PAG_UNIF)"
sheet_name2 = "REFI(HISTORICO)"
gc = connect_spreadsheets()
sh = gc.open_by_key(key)
worksheet = sh.worksheet(sheet_name)
worksheet2 = sh.worksheet(sheet_name2)

wks_data = worksheet.get_all_values()
#me conecto a la base de datos
cnxn = connect_database() #genero la conexion con la db

#inserto los datos de la spreadsheet en la db
cursor = cnxn.cursor()

#agarro el ultimo id pago 
query_ = "SELECT MAX(ID_PAGO) FROM ACTIVE.DBO.[PAGOS_UNIFICADOS] WHERE TIPO_PAGO ='REFI'"
max_id_pago = cursor.execute(query_).fetchall()[0][0] #el ultimo id_pagos


#me conecto a la base de datos
cols = wks_data[0]

#supervisores  gente a la que se le envia el mail en caso exitoso
supervisores = ['vanessa.fondado@mopc.cc','nicolas.rodriguez@mopc.cc','luis.castillo@mopc.cc','esteban.encina@mopc.cc','Nicolas.Caruso@mopc.cc']

fmt = cellFormat(backgroundColor=color(0, 1, 0) )
resultados= []

#inserto fila por fila en la base de datos
for i in range(1,len(wks_data)):
    
    values = wks_data[i]
    fecha_pago = datetime.strptime(values[cols.index("FECHA_PAGO")], "%Y-%m-%d").strftime("%Y-%d-%m %H:%M:%S")
    values[cols.index('FECHA_CARGA')] = time.strftime("%Y-%d-%m %H:%M:%S", time.localtime()) # fecha carga el tiempo actual
    values[cols.index('ID_PAGO')] = max_id_pago + i #le iserto el id-pago
    values[cols.index('EMPRESA')] = 'MO&PC' #este valor es fijo
    values[cols.index('TIPO_PAGO')] = 'REFI' #este valor es fijo
    values[cols.index("FECHA_PAGO")] = fecha_pago
    print(values)
    idmoroso = values[cols.index("IDMOROSO")]
    query = f"BEGIN IF NOT EXISTS (SELECT * FROM ACTIVE.DBO.PAGOS_UNIFICADOS WHERE TIPO_PAGO ='REFI' AND FECHA_PAGO = '{fecha_pago}' AND IDMOROSO = '{idmoroso}') BEGIN INSERT INTO ACTIVE.DBO.PAGOS_UNIFICADOS (FUENTE_PAGO, ID_PAGO, RUC, FECHA_PAGO, FECHA_REMESA, DESTINATARIO, IDCLIENTE, IDMOROSO, MONTO, HONORARIOS, FILTROA, FILTROB, FILTROC,DEUDA_TOTAL, DEUDA_VENCIDA, TIPO_PAGO, FECHA_CARGA, EMPRESA) VALUES {tuple(values)} END END"
    cursor.execute(query)
    cnxn.commit()
    format_cell_range(worksheet, f'{i+1}', fmt)
    print('Insert Finish!')
    #worksheet.delete_row(i+1)
    resultados.append(1)

print(resultados)
# me fijo si todas las filas fueron insertadas correctamente. Si no, se envia un mail automatico avisando
if sum(resultados) == len(wks_data) - 1:
            send_mail(subject = "REFIS: carga correcta",
        body = "Estimados, <br> Las REFIS fueron cargadas correctamente a las base de datos.<br><br>  Este correo se envía con frecuencias semanal y es automático. <br> Cualquier duda o consulta, comunicarse con ia@mopc.cc <br> Atte. <br> Equipo de Inteligencia Artificial" ,
receivers=supervisores, sender=None)
else: 
    send_mail(subject = "REFIS: error",
        body = "Estimados, <br> Las REFIS no fueron cargadas correctamente.<br><br>  Este correo se envía con frecuencias semanal y es automático. <br> Cualquier duda o consulta, comunicarse con ia@mopc.cc <br> Atte. <br> Equipo de Inteligencia Artificial" ,
receivers=['esteban.encina@mopc.cc','Nicolas.Caruso@mopc.cc'], sender=None)
    
cnxn.close()

['OP_BBVA', 453, '', '2021-28-05 00:00:00', '', 'FRANCES_2012', 'FRANCES_2012', 'DNI 26439133', '75700', '', '2 - Ag 2ra', 'RLESCANO', '', '', '', 'REFI', '2021-31-05 10:27:43', 'MO&PC']
Insert Finish!
['OP_BBVA', 454, '', '2021-28-05 00:00:00', '', 'FRANCES_2012', 'FRANCES_2012', 'DNI 36701303', '191200', '', '1 - Ag 1ra', 'EMACIEL', '', '', '', 'REFI', '2021-31-05 10:27:44', 'MO&PC']
Insert Finish!
['OP_BBVA', 455, '', '2021-28-05 00:00:00', '', 'FRANCES_2012', 'FRANCES_2012', 'DNI 34689197', '59100', '', '2 - Ag 2ra', 'SALAYETA', '', '', '', 'REFI', '2021-31-05 10:27:45', 'MO&PC']
Insert Finish!
['OP_BBVA', 456, '', '2021-28-05 00:00:00', '', 'FRANCES_2012', 'FRANCES_2012', 'DNI 31897655', '57000', '', '1 - Ag 1ra', 'CEZAPATA', '', '', '', 'REFI', '2021-31-05 10:27:46', 'MO&PC']
Insert Finish!
['OP_BBVA', 457, '', '2021-28-05 00:00:00', '', 'FRANCES_2012', 'FRANCES_2012', 'DNI 36140923', '48000', '', '1 - Ag 1ra', 'PERNICO', '', '', '', 'REFI', '2021-31-05 10:27:47', 'MO&PC']
Insert

In [41]:
cnxn.close()

In [29]:

from datetime import datetime
b = values[cols.index("FECHA_PAGO")]
a = datetime.strptime(b, "%Y-%m-%d").strftime("%Y-%d-%m %H:%M:%S")
a

'2020-22-05 00:00:00'

In [27]:
type(time.localtime())

time.struct_time

In [26]:
import os


from utils.functions_ia import *

cnxn = connect_database()
query = "SELECT * FROM ACTIVE.DBO.TABLERO_LATE_COL_ENRIQUE_V2"
df = pd.read_sql(query, cnxn).dropna(how = "all")
df.columns = [col.upper() for col in df.columns]

key = "1u-TSpwfKI8G4SQPjx0TZUFPR2xl0GejeYhkuvMs2rWY"

query= "SELECT * FROM ACTIVE.DBO.EVOLUCION_ENRIQUECIMIENTO_VIEW"
df= pd.read_sql(query, cnxn).dropna(how='all')
df.columns = [col.upper() for col in df.columns]
print(df.columns)
sheet_name= "Evolucion"


Index(['IDMOROSO', 'IDCLIENTE', 'FASIGNACION', 'FECHA_ENVIO_ENRIQUECIMIENTO',
       'PROVEEDOR', 'ENRIQ_90DIAS', 'AÑO_MES'],
      dtype='object')


In [34]:
import numpy as np
np.shape(df)

(45950, 7)

In [36]:
type(worksheet)

gspread.models.Worksheet